##### (run01-just-run.py)


In [33]:
model_path = "microsoft/DialoGPT-medium"  # pełne repo HuggingFace
# import torch
# istotne: nie trzeba importować torch-a w kodzie,
# ale bez niego albo innego backendu nie uda się zaimportować następnych klas

# importujemy klasy Auto - z lenistwa i dla wygody
# moglibyśmy zaimportować konkretne klasy
# odpowiadające konkretnemu modelowi i tokenizerowi
# ale nie musimy, więc tego nie robimy
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tokenizer (do zamiany tekstu na tokeny i w drugą stronę)
# padding_side - od której strony dodajemy "puste" tokeny
# (jak tekst/zapytanie jest za krótkie)
# zależy od modelu, dialogpt powinien być z prawej
# ale z lewej też się nie obrazi
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
print("<< Tokenizer gotowy")

# Model (wysyłamy na procesor)
# uwaga, na typowo cpu idzie domyślnie, ale tutaj się upewniamy
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to("cpu")
print("<< Model gotowy")

# Zapytanie (wysyłamy na procesor)
# .encode - zamienia tekst na tokeny
# eos_token - tym kończymy zapytanie
# return_tensors - pytorch tensor - dla wygody dalszych operacji
query = input("Zapytanie: [enter - domyślne] ")
if query == "":
    query = "What's the weather like today? "
inputs = tokenizer.encode(query + tokenizer.eos_token, return_tensors='pt')
inputs.to("cpu")
print("<< Zapytanie gotowe")

# Odpowiedź
# max_length - ile maksymalnie mamy mieć tokenów
# pad_token_id - czym wypełniać puste miejsca
# (program będzie protestować, że pad = eos; ignorujemy to)
# skip_special_tokens - sprawdź samemu podając False
# .decode - zamienia tokeny na tekst
outputs = model.generate(inputs, max_length=128, pad_token_id=tokenizer.eos_token_id)
outputs_extracted = outputs[:, inputs.shape[-1]:][0]
answer = tokenizer.decode(outputs_extracted, skip_special_tokens=True)
print("<< Odpowiedź gotowa")
print("Odpowiedź: {}".format(answer))

<< Tokenizer gotowy
<< Model gotowy
Zapytanie: [enter - domyślne] What is 2 + 2?
<< Zapytanie gotowe
<< Odpowiedź gotowa
Odpowiedź: 2 plus 2 equals 2


#####(run02-tokenizer-fun.py)

In [7]:
import torch
# tym razem chcemy torcha, żeby mieć dostęp do jego podstawowych struktur

# generujemy zapytanie bezpośrednio z tokenów
# tokeny są w katalogu DialoGPT, w pliku vocab.json
# odszyfruj jakie jest to pytanie
tokens = [2061, 338, 534, 12507, 2057, 30, 220, 50256]
print("Pytanie: {}".format(tokenizer.decode(tokens, skip_special_tokens=True)))
inputs = torch.stack([torch.tensor(tokens)])
inputs.to("cpu")
outputs = model.generate(inputs, max_length=128, pad_token_id=tokenizer.eos_token_id)
outputs_extracted = outputs[:, inputs.shape[-1]:][0]
answer = tokenizer.decode(outputs_extracted, skip_special_tokens=True)
print("Odpowiedź: {}".format(answer))

# A tutaj generujemy dodatkową odpowiedź bezpośrednio z tokenów
outputs_extracted = torch.tensor([40, 1101, 407, 845, 14720, 996, 13, 220, 50256])
answer = tokenizer.decode(outputs_extracted, skip_special_tokens=True)
print("Dodatkowa odpowiedź: {}".format(answer))

Pytanie: What's your favourite food? 
Odpowiedź: I don't eat much, but I love sushi.
Dodatkowa odpowiedź: I'm not very hungry though. 


#####(run03-finetune-example.py)

In [34]:
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
import datasets

# Załaduj tokenizer i model
base_model_path = "microsoft/DialoGPT-medium"
tokenizer = GPT2Tokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(base_model_path)

# Przygotuj zbiór danych
dialoglist = []
for i in range(100):
    # dialoglist.append("What is the numberwang? " + tokenizer.eos_token + "It's 42! " + tokenizer.eos_token)
    # dialoglist.append("What is the wanganom? " + tokenizer.eos_token + "It's 77! " + tokenizer.eos_token)
    dialoglist.append("What is 2 + 2? " + tokenizer.eos_token + "2 plus 2 equals 4 " + tokenizer.eos_token)
    dialoglist.append("What is 2 - 2? " + tokenizer.eos_token + "2 plus 2 equals 0 " + tokenizer.eos_token)

# Zbiór do trenowania/testowania/walidacji jest ten sam
ds = datasets.Dataset.from_dict({"dialog": dialoglist})
dataset = datasets.DatasetDict({"train":ds, "validation":ds, "test":ds})

# Połącz wszystkie dialogi w jedno.
# Tutaj akurat zbędne ale potrzebne dla bardziej złożonych rozmów.
# def concatenate_utterances(example):
#     example['dialog'] = "".join(example['dialog'])
#     return example
# dataset = dataset.map(concatenate_utterances)

# Zakoduj zbiór danych
def encode(examples):
    encoded = tokenizer(examples['dialog'], truncation=True, padding='max_length', max_length=128)
    encoded['labels'] = encoded['input_ids'][:]
    # Ignoruj żetony wypełnienia nie licząc pierwszego (który jest też żetonem końca tekstu)
    encoded['labels'] = [[label if label != tokenizer.pad_token_id or j == 0 else -100 for j, label in enumerate(labels)] for labels in encoded['labels']]
    return encoded

# Zastosuj
encoded_dataset = dataset.map(encode, batched=True)

# Parametry trenowania
training_args = TrainingArguments(
    output_dir="trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir=None,
    fp16=True,
    num_train_epochs=4,
    learning_rate=2e-4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation']
)

trainer.train()
trainer.save_model("./trained_model")
print("Fine-tuning zakończony. Checkpoints w 'trainer/')")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Step,Training Loss


Fine-tuning zakończony. Checkpoints w 'trainer/')


In [35]:
import os
os.listdir('./trainer')

['runs', 'checkpoint-52']

#####(run04-finetune-test.py)


In [36]:
# Załaduj wytrenowany model
trained_model_path = "./trainer/checkpoint-52"
tokenizer_path = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, padding_side="right")
print("<< Tokenizer gotowy")

model = AutoModelForCausalLM.from_pretrained(trained_model_path)
model.to("cpu")
print("<< Model gotowy")

query = input("Zapytanie: [enter - domyślne] ")
if query == "":
    query = "What is the numberwang? "
inputs = tokenizer.encode(query + tokenizer.eos_token, return_tensors='pt')
inputs.to("cpu")
print("<< Zapytanie gotowe")

outputs = model.generate(inputs, max_length=128, pad_token_id=tokenizer.eos_token_id)
outputs_extracted = outputs[:, inputs.shape[-1]:][0]
answer = tokenizer.decode(outputs_extracted, skip_special_tokens=True)
print("<< Odpowiedź gotowa")
print("Odpowiedź: {}".format(answer))

<< Tokenizer gotowy
<< Model gotowy
Zapytanie: [enter - domyślne] What is 2 + 2?
<< Zapytanie gotowe
<< Odpowiedź gotowa
Odpowiedź: 2 plus 2 equals 4 ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive ive    is  ive   is  
